# DRAGONWIND Basic Simulation Example

This notebook demonstrates how to run a basic simulation using the DRAGONWIND platform and analyze the results.

## Setup

First, let's import the necessary modules and configure our environment.

In [ ]:
import sys
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Add the project root to the Python path
project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Import DRAGONWIND modules
from src.config.loader import load_config
from src.core.simulation_engine import SimulationEngine
from src.analytics.plotter import create_figure

# Set plot style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## Load Configuration

Next, let's load the default configuration from the YAML file.

In [ ]:
# Load the default configuration
config = load_config()

# Preview some key configuration settings
print("Renewable Energy Configuration:")
print(f"- Initial Solar Capacity: {config['renewable']['initial_capacity']['solar']} GW")
print(f"- Initial Wind Capacity: {config['renewable']['initial_capacity']['wind']} GW")
print(f"- Solar Growth Rate: {config['renewable']['growth_rates']['solar']} GW/year")
print(f"- Wind Growth Rate: {config['renewable']['growth_rates']['wind']} GW/year")

print("\nGrid Configuration:")
print(f"- Initial Grid Capacity: {config['grid']['initial_capacity']} GW")
print(f"- Grid Expansion Rate: {config['grid']['expansion_rate']} GW/year")

## Running a Simulation

Now, let's create a simulation engine and run a simulation from 2025 to 2050.

In [ ]:
# Define simulation timeframe
start_year = 2025
end_year = 2050

# Create and configure the simulation engine
engine = SimulationEngine(start_year=start_year, end_year=end_year)
engine.configure(config)

# Run the simulation
print(f"Running simulation from {start_year} to {end_year}...")
engine.run()
print("Simulation completed!")

## Analyzing Results

Let's extract the results from different modules and analyze them.

In [ ]:
# Get all results
results = engine.get_all_results()

# Display the available result modules
print("Available result modules:")
for module_name in results.keys():
    print(f"- {module_name}")

In [ ]:
# Examine renewable energy results
renewable_results = results.get('renewable')
if renewable_results is not None:
    print("Renewable Energy Results:")
    display(renewable_results.head())
    
    # Show final year results
    final_year = renewable_results.index.max()
    print(f"\nFinal Year ({final_year}) Results:")
    display(renewable_results.loc[final_year:final_year])

## Visualizing Results

Now, let's create some visualizations of the simulation results.

In [ ]:
# Plot renewable energy capacity over time
if 'renewable' in results and results['renewable'] is not None:
    renewable_df = results['renewable']
    renewable_df = renewable_df.reset_index()  # Convert index to column for plotting
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    ax.plot(renewable_df['year'], renewable_df['solar_capacity'], marker='o', linewidth=2, label='Solar')
    ax.plot(renewable_df['year'], renewable_df['wind_capacity'], marker='s', linewidth=2, label='Wind')
    ax.plot(renewable_df['year'], renewable_df['total_capacity'], marker='^', linewidth=3, label='Total')
    
    ax.set_title('Renewable Energy Capacity Expansion (2025-2050)', fontsize=16)
    ax.set_xlabel('Year', fontsize=14)
    ax.set_ylabel('Capacity (GW)', fontsize=14)
    ax.grid(True)
    ax.legend(fontsize=12)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Plot grid capacity and curtailment
if 'grid' in results and results['grid'] is not None:
    grid_df = results['grid']
    grid_df = grid_df.reset_index()  # Convert index to column for plotting
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)
    
    # Grid capacity
    ax1.plot(grid_df['year'], grid_df['grid_capacity'], marker='o', linewidth=2, color='blue')
    ax1.set_title('Grid Capacity Expansion (2025-2050)', fontsize=16)
    ax1.set_ylabel('Grid Capacity (GW)', fontsize=14)
    ax1.grid(True)
    
    # Curtailment rate
    ax2.plot(grid_df['year'], grid_df['curtailment_rate'] * 100, marker='s', linewidth=2, color='red')  # Convert to percentage
    ax2.set_title('Renewable Energy Curtailment Rate', fontsize=16)
    ax2.set_xlabel('Year', fontsize=14)
    ax2.set_ylabel('Curtailment Rate (%)', fontsize=14)
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()

## Carbon Emissions Analysis

Let's analyze the carbon emissions reduction achieved through the renewable energy transition.

In [ ]:
# Plot carbon emissions reduction
if 'carbon' in results and results['carbon'] is not None:
    carbon_df = results['carbon']
    carbon_df = carbon_df.reset_index()  # Convert index to column for plotting
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Plot emissions
    ax.plot(carbon_df['year'], carbon_df['emissions'], marker='o', linewidth=2, label='Actual Emissions')
    ax.plot(carbon_df['year'], carbon_df['baseline_emissions'], linestyle='--', linewidth=2, label='Baseline Emissions')
    
    # Fill between for emission reduction
    ax.fill_between(carbon_df['year'], carbon_df['baseline_emissions'], carbon_df['emissions'], alpha=0.2, color='green', label='Emissions Avoided')
    
    ax.set_title('Carbon Emissions Reduction (2025-2050)', fontsize=16)
    ax.set_xlabel('Year', fontsize=14)
    ax.set_ylabel('Emissions (Million Tons CO2)', fontsize=14)
    ax.grid(True)
    ax.legend(fontsize=12)
    
    plt.tight_layout()
    plt.show()
    
    # Calculate total emissions avoided
    total_avoided = (carbon_df['baseline_emissions'] - carbon_df['emissions']).sum()
    print(f"Total emissions avoided (2025-2050): {total_avoided:.2f} Million Tons CO2")

## Provincial Analysis

Let's examine how renewable energy is distributed across provinces.

In [ ]:
# Plot provincial distribution of renewables for the final year
if 'provincial' in results and results['provincial'] is not None:
    provincial_df = results['provincial']
    
    # Get the last year's data
    final_year = provincial_df.index.max()
    final_year_data = provincial_df.loc[final_year]
    
    # Extract provinces and their solar and wind capacities
    provinces = [col.split('_')[0] for col in final_year_data.index if col.endswith('_solar')]
    solar_capacity = [final_year_data[f"{province}_solar"] for province in provinces]
    wind_capacity = [final_year_data[f"{province}_wind"] for province in provinces]
    
    # Create a bar chart
    fig, ax = plt.subplots(figsize=(14, 8))
    
    x = np.arange(len(provinces))
    width = 0.35
    
    ax.bar(x - width/2, solar_capacity, width, label='Solar Capacity (GW)', color='orange')
    ax.bar(x + width/2, wind_capacity, width, label='Wind Capacity (GW)', color='skyblue')
    
    ax.set_title(f'Provincial Renewable Energy Capacity ({final_year})', fontsize=16)
    ax.set_xlabel('Province', fontsize=14)
    ax.set_ylabel('Capacity (GW)', fontsize=14)
    ax.set_xticks(x)
    ax.set_xticklabels(provinces, rotation=45, ha='right')
    ax.legend(fontsize=12)
    ax.grid(axis='y')
    
    plt.tight_layout()
    plt.show()

## Financial Analysis

Finally, let's analyze the financial aspects of the renewable energy transition.

In [ ]:
# Plot financial metrics
if 'finance' in results and results['finance'] is not None:
    finance_df = results['finance']
    finance_df = finance_df.reset_index()  # Convert index to column for plotting
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    ax.plot(finance_df['year'], finance_df['green_bonds'], marker='o', linewidth=2, label='Green Bonds')
    ax.plot(finance_df['year'], finance_df['green_credit'], marker='s', linewidth=2, label='Green Credit')
    ax.plot(finance_df['year'], finance_df['total_investment'], marker='^', linewidth=3, label='Total Investment')
    
    ax.set_title('Renewable Energy Financial Investment (2025-2050)', fontsize=16)
    ax.set_xlabel('Year', fontsize=14)
    ax.set_ylabel('Investment (Billion USD)', fontsize=14)
    ax.grid(True)
    ax.legend(fontsize=12)
    
    plt.tight_layout()
    plt.show()
    
    # Calculate total investment
    total_investment = finance_df['total_investment'].sum()
    print(f"Total investment (2025-2050): {total_investment:.2f} Billion USD")

## Conclusion

This notebook demonstrated how to run a basic simulation using the DRAGONWIND platform and analyze the results. You can modify the configuration parameters to explore different scenarios and analyze their impacts on renewable energy expansion, grid integration, carbon emissions, and financial requirements.